LAB Python and SQL

In [1]:
# prep: import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input

In [2]:
password = getpass.getpass()

 ········


1. Establish a connection between Python and the Sakila database.

In [3]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [4]:
query = '''select * from film;'''

In [7]:
data = pd.read_sql_query(query, engine)

2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

--engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.

--month: an integer representing the month for which rental data is to be retrieved.

--year: an integer representing the year for which rental data is to be retrieved.

The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [96]:
def rentals(engine: object, f_month: int, f_year: int) -> pd.DataFrame:
    '''
    this function will return all the columns in the rental table filtered by month and year
    inputs:
    engine: object
    f_month: int: selected month to filter data
    f_year: int: selected year to filter data
    output:
    pd.DataFrame
    '''
    
    query = f'''
        SELECT *
        FROM rental
        WHERE YEAR(rental_date) = {f_year} AND MONTH(rental_date) = {f_month};
        '''
    data_f = pd.read_sql_query(query, engine)
#   data_f.set_index('rental_id', inplace=True) # to avoid having index duplicity
    return data_f

In [104]:
query_test = rentals(engine, 5, 2005)
#query_test.shape

(1156, 7)

3. Develop a Python function called rental_count_month that takes the 

-DataFrame provided by rentals_month as input along with the 

-month and year and 

-returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [126]:
def rental_count_month(df: pd.DataFrame, f_month: int, f_year: int) -> pd.DataFrame:
    '''
    '''
    
    col_name = f'rentals_{f_month}_{f_year}' #set new column name string using F-strings
    
    df_grouped = df.groupby('customer_id').agg({'rental_id': 'count'}) # summarize data
    
    df_renamed = df_grouped.rename(columns={'rental_id': col_name}) #rename column

    df_sorted = df_renamed.sort_values(by=col_name, ascending=False)
        
    return df_sorted

In [127]:
rental_count_month(query_test, 5, 2005)

,rentals_5_2005
customer_id,
197,8
109,7
506,7
53,6
269,6
...,...
352,1
353,1
355,1


In [139]:
# additional step: define function to combine previous two functions
def rental_count_month_combined(engine: object, f_month: int, f_year: int) -> pd.DataFrame:
    '''
    this functions combines the previous two fn (rentals() and rental_count_month()) 
    as they share the month/year arguments and the output DataFrame of one is the input of the 2nd one.
    inputs:
    engine: object
    f_month: int: selected month to filter data
    f_year: int: selected year to filter data
    output:
    pd.DataFrame
    '''
    result_rentals=rentals(engine, f_month, f_year)
        
    result_count=rental_count_month(result_rentals, f_month, f_year)

    return result_count

4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [152]:
def compare_rentals(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    '''
    the function returns a combined DataFrame with a new 'difference' column, 
    which is the difference between the number of rentals in the two months.
    inputs:
    df1 first dataframe to compare
    df2 2nd dataframe to compate
    output:
    df combined DataFrame with a new 'difference' column
    '''
    # combine both dataframes
    merged_df = df1.merge(df2, on='customer_id')
    merged_df['difference'] = merged_df['rentals_2_2006'] - merged_df['rentals_5_2005']
    return merged_df
    

In [143]:
#create test dataframes

rentals_05_2005=rental_count_month_combined(engine, 5, 2005)
rentals_02_2006=rental_count_month_combined(engine, 2, 2006)

In [153]:
compare_rentals(rentals_05_2005, rentals_02_2006)

,rentals_5_2005,rentals_2_2006,difference
customer_id,,,
53,6,2,-4
269,6,2,-4
245,6,1,-5
251,6,1,-5
596,6,1,-5
...,...,...,...
324,1,1,0
327,1,1,0
349,1,1,0
